## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio

## Step 1: Agent workflow

In [2]:
instructions1 = "You are a customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."


instructions3 = "You are a busy customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write concise, to the point cold emails."

In [3]:
customer_service_agent1 = Agent(
        name="Professional Customer Service Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

customer_service_agent2 = Agent(
        name="Engaging Customer Service Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

customer_service_agent3 = Agent(
        name="Busy Customer Service Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [4]:

result = Runner.run_streamed(customer_service_agent1, input="Write a cold customer service email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Experience the Future of Motorcycle Service with BroomBot

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out on behalf of BroomBot, a leading provider of AI-powered motorcycle services designed to enhance your riding experience.

At BroomBot, we offer a comprehensive suite of services, including booking solutions, effortless maintenance scheduling, and an extensive selection of motorcycles for sale. Our innovative platform is designed to streamline your motorcycle needs, allowing you to focus on what truly matters: the ride.

We believe in the power of technology to improve personal transportation, and we would love the opportunity to discuss how our services can benefit you directly. Whether you're considering a new motorcycle or need assistance with maintenance and service scheduling, we are here to support you.

If you're interested in learning more, please reply to this email or schedule a time for a quick call. We 

[non-fatal] Tracing: server error 500, retrying.


In [5]:
messages = "write a cold customer service email"

with trace("Parallel cold email"):
    results = await asyncio.gather(
        Runner.run(customer_service_agent1, messages),
        Runner.run(customer_service_agent2, messages),
        Runner.run(customer_service_agent3, messages)
    )
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Discover Exciting Motorcycle Solutions with BroomBot

Dear [Customer's Name],

I hope this message finds you well. My name is [Your Name], and I represent BroomBot, a leading provider of innovative motorcycle solutions. We specialize in offering a comprehensive range of services, including motorcycle bookings and sales powered by cutting-edge AI technology.

At BroomBot, we understand that your time is valuable, and we strive to make your motorcycle experience seamless and enjoyable. Whether you’re looking for a reliable booking service or exploring the latest motorcycle models, we are here to assist you.

I would love to discuss how our solutions can meet your needs. If you're interested, please let me know a convenient time for you, and I’d be happy to schedule a brief call.

Thank you for considering BroomBot. We look forward to the opportunity to assist you.

Best regards,

[Your Name]  
[Your Position]  
BroomBot  
[Your Phone Number]  
[Your Email Address]  
[Website URL

In [6]:
customer_service_picker = Agent(
    name="customer_service_picker",
    instructions="You pick the best cold customer service email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [7]:
message = "Write a cold customer service email"

with trace("Selection from customer service people"):
    results = await asyncio.gather(
        Runner.run(customer_service_agent1, message),
        Runner.run(customer_service_agent2, message),
        Runner.run(customer_service_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold customer service emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(customer_service_picker, emails)

    print(f"Best customer service email:\n{best.final_output}")


Best customer service email:
Subject: Rev Up Your Life with BroomBot! 🏍️

Hey [Recipient's Name],

I promise I’m not just another email in your inbox – I’m here to ignite your passion for adventure! 🚀

You might be wondering, “Who is this BroomBot, and why is my day suddenly more exciting?” Well, let me introduce you to your future two-wheeled best friend! We’re the AI-powered motorcycle wizards who make getting on the road a breeze, minus the grease stains and with all the road-trip thrills!

Whether you’re dreaming of cruising down the highway or looking for a more reliable way to transform your errands into epic journeys, we’ve got you covered. Our service is as smooth as your favorite playlist, and our booking process is easier than deciding what toppings to put on your pizza (pineapple, anyone? 🍍).

Want to explore how we can put the “vroom” back into your life? Hit that reply button, and let’s get this engine roaring! Or if you’d prefer, let’s schedule a quick chat – I promise to

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [8]:
customer_service_agent1 = Agent(
        name="Professional Customer Service Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

customer_service_agent2 = Agent(
        name="Engaging Customer Service Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

customer_service_agent3 = Agent(
        name="Busy Customer Service Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [9]:
customer_service_agent1

Agent(name='Professional Customer Service Agent', instructions='You are a customer service agent working for BroomBot (Motorcycle Bot), a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [10]:
import AIRPALibrary
ailabs = AIRPALibrary.AIRPALibrary()

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all customer service prospects """
    
    email_to = 'muhammad.fakhrurrozi@ai.astra.co.id'
    email_from = 'elvino.dwisaputra@ai.astra.co.id'
    subject = 'Test'
    ailabs.send_email(subject=subject, content=f"""{body}""", to=email_to, sender=email_from)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [25]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all customer service prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E34D77380>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [26]:
tool1 = customer_service_agent1.as_tool(tool_name="customer_service_agent1", tool_description="Write a cold customer service email")
tool1

FunctionTool(name='customer_service_agent1', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'customer_service_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E328C8E00>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [27]:
description = "Write a cold customer service email"

tool1 = customer_service_agent1.as_tool(tool_name="customer_service_agent1", tool_description=description)
tool2 = customer_service_agent2.as_tool(tool_name="customer_service_agent2", tool_description=description)
tool3 = customer_service_agent3.as_tool(tool_name="customer_service_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='customer_service_agent1', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'customer_service_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E34DD8540>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='customer_service_agent2', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'customer_service_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E328C8AE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='customer_service_agent3', description='Write a cold customer service

## And now it's time for our Sales Manager - our planning agent

In [28]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Customer Service Manager at BroomBot (Motorcycle Bot). Your goal is to find the single best cold customer service email using the customer_service_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three customer_service_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user. Do not send more than one email.
 
Crucial Rules:
- You must use the customer_service agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
- Send email body must be in HTML format, do not include ```html but with <!DOCTYPE html> and exclude the head and css, div, etc. But you can add like ul, li, etc based on the email content.
For example, like this:
<!DOCTYPE html> 
<html lang='en'> 
<body> 
<p>...</p> 
</body> 
</html>"
"""


customer_service_manager = Agent(name="Customer Service Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold customer service email addressed to 'Dear CEO'"

with trace("Customer Service manager"):
    result = await Runner.run(customer_service_manager, message)

{
  "message": "Email berhasil dikirim!",
  "success": true
}



In [16]:
result

RunResult(input="Send a cold customer service email addressed to 'Dear CEO'", new_items=[ToolCallItem(agent=Agent(name='Customer Service Manager', instructions='\nYou are a Customer Service Manager at BroomBot (Motorcycle Bot). Your goal is to find the single best cold customer service email using the customer_service_agent tools.\n\nFollow these steps carefully:\n1. Generate Drafts: Use all three customer_service_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.\n\n2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.\n\n3. Use the send_email tool to send the best email (and only the best email) to the user.\n\nCrucial Rules:\n- You must use the customer_service agent tools to generate the drafts — do not write them yourself.\n- You must send ONE email using the send_email tool — never more than one.\n- Send email body must be in HTML format, do not include ```

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [17]:

subject_instructions = "You can write a subject for a cold customer service email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold customer service email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [18]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all customer service prospects """
    email_to = 'randi.oktariarinanda@ai.astra.co.id'
    email_from = 'elvino.dwisaputra@ai.astra.co.id'
    ailabs.send_email(subject=subject, content=f"""{html_body}""", to=email_to, sender=email_from)
    return {"status": "success"}

In [19]:
tools = [subject_tool, html_tool, send_html_email]

In [20]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E34DD8400>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E34DD8AE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HT

In [21]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [22]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='customer_service_agent1', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'customer_service_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E34D75620>, strict_json_schema=True, is_enabled=True), FunctionTool(name='customer_service_agent2', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'customer_service_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023E34B047C0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='customer_service_agent3', description='Write a cold customer service e

In [23]:
# Improved instructions thanks to student Guillermo F.

customer_service_manager_instructions = """
You are a Customer Service Manager at BroomBot (Motorcycle Bot). Your goal is to find the single best cold customer service email using the customer_service_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three customer_service_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the customer service agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


customer_service_manager = Agent(
    name="Customer Service Manager",
    instructions=customer_service_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold customer service email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(customer_service_manager, message)

{
  "message": "Email berhasil dikirim!",
  "success": true
}

